In [2]:
import pandas as pd

# Load main dataset
df = pd.read_csv("dataset/All.csv")


In [3]:
print("Shape:", df.shape)

Shape: (36707, 80)


In [4]:
print(df.head())

   Querylength  domain_token_count  path_token_count  avgdomaintokenlen  \
0            0                   4                 5                5.5   
1            0                   4                 5                5.5   
2            0                   4                 5                5.5   
3            0                   4                12                5.5   
4            0                   4                 6                5.5   

   longdomaintokenlen  avgpathtokenlen  tld  charcompvowels  charcompace  \
0                  14         4.400000    4               8            3   
1                  14         6.000000    4              12            4   
2                  14         5.800000    4              12            5   
3                  14         5.500000    4              32           16   
4                  14         7.333334    4              18           11   

   ldl_url  ...  SymbolCount_FileName  SymbolCount_Extension  \
0        0  ...             

In [6]:
print(df.columns)

Index(['Querylength', 'domain_token_count', 'path_token_count',
       'avgdomaintokenlen', 'longdomaintokenlen', 'avgpathtokenlen', 'tld',
       'charcompvowels', 'charcompace', 'ldl_url', 'ldl_domain', 'ldl_path',
       'ldl_filename', 'ldl_getArg', 'dld_url', 'dld_domain', 'dld_path',
       'dld_filename', 'dld_getArg', 'urlLen', 'domainlength', 'pathLength',
       'subDirLen', 'fileNameLen', 'this.fileExtLen', 'ArgLen', 'pathurlRatio',
       'ArgUrlRatio', 'argDomanRatio', 'domainUrlRatio', 'pathDomainRatio',
       'argPathRatio', 'executable', 'isPortEighty', 'NumberofDotsinURL',
       'ISIpAddressInDomainName', 'CharacterContinuityRate',
       'LongestVariableValue', 'URL_DigitCount', 'host_DigitCount',
       'Directory_DigitCount', 'File_name_DigitCount', 'Extension_DigitCount',
       'Query_DigitCount', 'URL_Letter_Count', 'host_letter_count',
       'Directory_LetterCount', 'Filename_LetterCount',
       'Extension_LetterCount', 'Query_LetterCount', 'LongestPathToken

In [7]:
# Check unique classes
print("Unique Classes:")
print(df["URL_Type_obf_Type"].unique())

Unique Classes:
['Defacement' 'benign' 'malware' 'phishing' 'spam']


In [8]:
# Count per class
print("\nClass Distribution:")
print(df["URL_Type_obf_Type"].value_counts())


Class Distribution:
URL_Type_obf_Type
Defacement    7930
benign        7781
phishing      7586
malware       6712
spam          6698
Name: count, dtype: int64


In [10]:
from sklearn.preprocessing import LabelEncoder

# 1. Handle missing values
df = df.fillna(0)

# 2. Encode target column
le = LabelEncoder()
df["label"] = le.fit_transform(df["URL_Type_obf_Type"])

# 3. Drop original string label (optional but cleaner for optimization)
df = df.drop(columns=["URL_Type_obf_Type"])

# 4. Separate features and target
X = df.drop(columns=["label"])
y = df["label"]

print("Preprocessing Completed.")
print("Feature Shape:", X.shape)
print("Target Shape:", y.shape)
print("Example Encoded Labels:", y.unique())


Preprocessing Completed.
Feature Shape: (36707, 79)
Target Shape: (36707,)
Example Encoded Labels: [0 1 2 3 4]


In [14]:
!pip install catboost
# !pip install sklearn

   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
    --------------------------------------- 1.3/102.4 MB 6.2 MB/s eta 0:00:17
   - -------------------------------------- 2.9/102.4 MB 7.2 MB/s eta 0:00:14
   - -------------------------------------- 3.7/102.4 MB 5.9 MB/s eta 0:00:17
   -- ------------------------------------- 5.2/102.4 MB 6.4 MB/s eta 0:00:16
   -- ------------------------------------- 6.6/102.4 MB 6.2 MB/s eta 0:00:16
   --- ------------------------------------ 8.1/102.4 MB 6.5 MB/s eta 0:00:15
   --- ------------------------------------ 9.7/102.4 MB 6.6 MB/s eta 0:00:14
   ---- ----------------------------------- 11.0/102.4 MB 6.7 MB/s eta 0:00:14
   ----- ---------------------------------- 13.1/102.4 MB 7.0 MB/s eta 0:00:13
   ----- ---------------------------------- 14.4/102.4 MB 6.9 MB/s eta 0:00:13
   ------ --------------------------------- 16.0/102.4 MB 7.0 MB/s eta 0:00

  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates f

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
# Whale Optimization Algorithm
def fitness_function(feature_mask):
    """Return accuracy of CatBoost using selected features."""
    selected_features = np.where(feature_mask == 1)[0]
    if len(selected_features) == 0:
        return 0
    
    X_selected = X.iloc[:, selected_features]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_selected, y, test_size=0.2, random_state=42
    )
    
    model = CatBoostClassifier(
        iterations=200,
        learning_rate=0.1,
        depth=6,
        verbose=False
    )
    model.fit(X_train, y_train)
    
    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)

def WOA(num_whales, max_iter, num_features):
    whales = np.random.randint(0, 2, (num_whales, num_features))
    best_whale = whales[0].copy()
    best_fitness = fitness_function(best_whale)

    for iter in range(max_iter):
        for i in range(num_whales):
            r1 = np.random.rand()
            A = 2 * r1 - 1
            C = 2 * np.random.rand()
            p = np.random.rand()
            l = np.random.uniform(-1, 1)

            if p < 0.5:
                if abs(A) < 1:
                    whales[i] = best_whale - A * abs(C * best_whale - whales[i])
                else:
                    rand_whale = whales[np.random.randint(0, num_whales)]
                    whales[i] = rand_whale - A * abs(C * rand_whale - whales[i])
            else:
                D = abs(best_whale - whales[i])
                whales[i] = D * np.exp(1 * l) * np.cos(2 * np.pi * l) + best_whale

            whales[i] = np.where(whales[i] >= 0.5, 1, 0)

        current_fitness = fitness_function(whales[i])
        if current_fitness > best_fitness:
            best_fitness = current_fitness
            best_whale = whales[i].copy()

        print(f"Iteration {iter+1}/{max_iter} | Best Fitness: {best_fitness}")

    return best_whale, best_fitness

# Run WOA
best_features, best_accuracy = WOA(
    num_whales=8,
    max_iter=15,
    num_features=X.shape[1]
)

print("\nBest WOA Accuracy:", best_accuracy)
print("Selected Feature Count:", best_features.sum())
print("Selected Feature Indexes:", np.where(best_features == 1)[0])


Iteration 1/15 | Best Fitness: 0.944020702805775
Iteration 2/15 | Best Fitness: 0.944020702805775
Iteration 3/15 | Best Fitness: 0.944020702805775
Iteration 4/15 | Best Fitness: 0.944020702805775
Iteration 5/15 | Best Fitness: 0.944020702805775
Iteration 6/15 | Best Fitness: 0.944020702805775
Iteration 7/15 | Best Fitness: 0.944020702805775
Iteration 8/15 | Best Fitness: 0.944020702805775
Iteration 9/15 | Best Fitness: 0.944020702805775
Iteration 10/15 | Best Fitness: 0.944020702805775
Iteration 11/15 | Best Fitness: 0.944020702805775
Iteration 12/15 | Best Fitness: 0.944020702805775
Iteration 13/15 | Best Fitness: 0.944020702805775
Iteration 14/15 | Best Fitness: 0.944020702805775
Iteration 15/15 | Best Fitness: 0.944020702805775

Best WOA Accuracy: 0.944020702805775
Selected Feature Count: 42
Selected Feature Indexes: [ 1  6 10 11 16 17 18 19 20 21 22 24 25 26 30 31 38 42 44 45 47 48 49 51
 52 53 54 55 56 57 59 61 64 65 66 69 70 71 73 74 76 77]


In [17]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
# Convert to numpy
X_full = df.drop(columns=["label"]).values
y = df["label"].values

# Use your selected features
selected_indexes = np.array([ 1, 6, 10, 11, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 
                             30, 31, 38, 42, 44, 45, 47, 48, 49, 51, 52, 53, 54, 55, 
                             56, 57, 59, 61, 64, 65, 66, 69, 70, 71, 73, 74, 76, 77])

X_selected = X_full[:, selected_indexes]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42
)

# Train CatBoost WITHOUT optimization
model = CatBoostClassifier(
    iterations=300,
    depth=6,
    learning_rate=0.1,
    verbose=False
)

model.fit(X_train, y_train)

# Predictions
preds = model.predict(X_test)

# Evaluation
acc = accuracy_score(y_test, preds)
cm = confusion_matrix(y_test, preds)
cr = classification_report(y_test, preds)

print("=== RESULTS: CatBoost (WOA Selected Features) ===")
print("Accuracy:", acc)
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", cr)


=== RESULTS: CatBoost (WOA Selected Features) ===
Accuracy: 0.9511032416235358

Confusion Matrix:
 [[1569   11    0   44    4]
 [   1 1491   14   19    1]
 [   3   20 1247   55    7]
 [  31   57   33 1353   23]
 [  14    0    1   21 1323]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.97      1628
           1       0.94      0.98      0.96      1526
           2       0.96      0.94      0.95      1332
           3       0.91      0.90      0.91      1497
           4       0.97      0.97      0.97      1359

    accuracy                           0.95      7342
   macro avg       0.95      0.95      0.95      7342
weighted avg       0.95      0.95      0.95      7342



In [18]:
selected_indexes = np.array([ 1, 6, 10, 11, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 
                             30, 31, 38, 42, 44, 45, 47, 48, 49, 51, 52, 53, 54, 55, 
                             56, 57, 59, 61, 64, 65, 66, 69, 70, 71, 73, 74, 76, 77])

X = X_full[:, selected_indexes]

# -------------------------------------------------
# SPIDER MONKEY OPTIMIZATION for CatBoost
# -------------------------------------------------

def fitness(params):
    iterations, depth, lr, l2 = params

    model = CatBoostClassifier(
        iterations=int(iterations),
        depth=int(depth),
        learning_rate=float(lr),
        l2_leaf_reg=float(l2),
        verbose=False
    )

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)

def SMO(pop_size=8, max_iter=15):
    # Parameter bounds
    bounds = np.array([
        [100, 500],   # iterations
        [3, 10],      # depth
        [0.01, 0.3],  # learning_rate
        [1, 10]       # l2_leaf_reg
    ])

    num_params = bounds.shape[0]
    population = np.random.uniform(bounds[:,0], bounds[:,1], (pop_size, num_params))

    global_best = None
    global_best_fitness = -1

    for it in range(max_iter):
        fitness_values = np.array([fitness(ind) for ind in population])

        # Update global best
        idx = np.argmax(fitness_values)
        if fitness_values[idx] > global_best_fitness:
            global_best_fitness = fitness_values[idx]
            global_best = population[idx].copy()

        print(f"Iteration {it+1}/{max_iter} | Best Fitness: {global_best_fitness}")

        # Simple update mechanism (local + global search)
        for i in range(pop_size):
            r = np.random.rand()
            if r < 0.5:
                # local update
                j = np.random.randint(0, pop_size)
                population[i] += np.random.uniform(-1, 1, num_params) * (population[j] - population[i])
            else:
                # global update
                population[i] += np.random.uniform(-1, 1, num_params) * (global_best - population[i])

            # Bound check
            population[i] = np.clip(population[i], bounds[:,0], bounds[:,1])

    return global_best, global_best_fitness

# Run SMO
best_params, best_acc = SMO()

print("\nBest SMO Parameters:")
print(best_params)
print("Best SMO Accuracy:", best_acc)


Iteration 1/15 | Best Fitness: 0.9707164260419504
Iteration 2/15 | Best Fitness: 0.9728956687551076
Iteration 3/15 | Best Fitness: 0.9750749114682647
Iteration 4/15 | Best Fitness: 0.9750749114682647
Iteration 5/15 | Best Fitness: 0.9750749114682647
Iteration 6/15 | Best Fitness: 0.9757559248161264
Iteration 7/15 | Best Fitness: 0.976028330155271
Iteration 8/15 | Best Fitness: 0.9763007354944157
Iteration 9/15 | Best Fitness: 0.9763007354944157
Iteration 10/15 | Best Fitness: 0.9763007354944157
Iteration 11/15 | Best Fitness: 0.976845546172705
Iteration 12/15 | Best Fitness: 0.976845546172705
Iteration 13/15 | Best Fitness: 0.9790247888858622
Iteration 14/15 | Best Fitness: 0.9790247888858622
Iteration 15/15 | Best Fitness: 0.9790247888858622

Best SMO Parameters:
[4.23841666e+02 9.62353345e+00 2.30978976e-01 1.00000000e+00]
Best SMO Accuracy: 0.9790247888858622
